In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext tensorboard

In [3]:
import gym
from matplotlib import pyplot as plt
from agents import *
from tqdm.notebook import tqdm
import numpy as np
from wrappers import Recorder
from torch.utils.tensorboard import SummaryWriter

/opt/anaconda3/envs/week6/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


## Cart Pole

[Description](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py)

In [4]:
env = Recorder(gym.make("CartPole-v1"))
writer = SummaryWriter()

In [5]:
env.action_space.n

2

In [6]:
env.reset()

array([ 0.03574043, -0.04664144,  0.02296554, -0.03243444], dtype=float32)

In [9]:
network = ApproxQLearningNetwork(n_features=4, n_actions=2, hidden_dimensions=128)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
deep_q_agent = ApproxQAgent(network, optimizer, eps=0.05)

In [10]:
batch_num = 0.
for epoch in tqdm(range(100_000)):
    total_reward = 0.
    n_games = 10
    n_wins = 0
    episodes: List[Episode] = []
    for game_id in range(n_games):
        state = env.reset()
        is_done = False
        game_reward = 0
        while not is_done:
            action = deep_q_agent.act(state, greedy=False)
            state, reward, is_done, info = env.step(action)
            game_reward += reward
        if game_reward > 200:
            n_wins += 1
        total_reward += game_reward
        episodes.extend(env.get_record())
    writer.add_scalar("Game/wins_out_of_10", n_wins, epoch)
    np.random.shuffle(episodes)
    batch_sz = 128
    for batch_ptr in range(0, len(episodes), batch_sz):
        batch = episodes[batch_ptr:batch_ptr + batch_sz]
        loss = deep_q_agent.train_on_batch(batch)
        writer.add_scalar("Loss/train", loss, batch_num)
        batch_num += 1
    writer.add_scalar("Avg/reward", total_reward / n_games, epoch)


  0%|          | 0/100000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
len(env.get_record())

135